In [ ]:

from dotenv import load_dotenv
import os
import sys
import json
import pandas as pd
import time
import datetime
load_dotenv()

path = os.path.abspath('')
module_path = os.path.join(path,'..\..')
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path+"\\functions")


from dataset_utils import DatasetEvaluator

In [2]:
DEFAULT_PATH = "results/"
EXPERIMENT_RESULT_PATH = f"{DEFAULT_PATH}sql_query_chain_result_files.txt"
EXPERIMENT = "SQLQueryChain"
TARGET_PATH = f"evaluator_results/{EXPERIMENT}_results.csv"
TARGET_PATH_FK = f"evaluator_results/{EXPERIMENT}_foreign_key_results.csv"


Funções úteis

In [3]:
def save_queries(queries, FILE_NAME_PATH):
    data = {"queries":queries}
    with open(FILE_NAME_PATH, "w") as arquivo_json:
        json.dump(data, arquivo_json, indent=4) 

def read_queries(FILE_NAME_PATH):
    with open(FILE_NAME_PATH, encoding="utf-8", errors="ignore") as json_data:
        data = json.load(json_data, strict=False)
    queries = data["queries"]
    return queries

def read_files():
    with open(EXPERIMENT_RESULT_PATH, "r") as f:
        files = f.readlines()
    files = [f"{DEFAULT_PATH}{x.strip()}" for x in files]
    return files

def get_schema(FILE_NAME_PATH):
    if "mondial_gpt" in FILE_NAME_PATH:
        return "mondial_gpt"
    elif "mondial" in FILE_NAME_PATH:
        return "mondial"

    return ""

def get_count_query_type(queries):
    count_types = {}
    for item in queries:
        type = item["type"]
        if type in count_types:
            count_types[type] += 1
        else:
            count_types[type] = 1
    count_types['total'] = len(queries)
    return count_types

def get_dataset_evaluator(FILE_NAME_PATH):
    SCHEMA = get_schema(FILE_NAME_PATH)  
    PREFIX = SCHEMA  
    if SCHEMA == "mondial_gpt":
        PREFIX = "mondial"
    print(PREFIX)
    print(SCHEMA)
    dataset_eval = DatasetEvaluator(dataset_file_path=f"../../datasets/{PREFIX}/{PREFIX}_dataset.json",
                                dataset_tables_path=f"../../datasets/{PREFIX}/result_tables/",
                                db_connection_file=f"../../datasets/{SCHEMA}_db_connection.json",
                                dataset_name=SCHEMA)
    return dataset_eval

def get_files_in_folder():
    files = []
    for r, d, f in os.walk(DEFAULT_PATH):
        for file in f:
            if '.json' in file and 'erro' not in file:
                files.append(os.path.join(r, file))
    return sorted(files)

def evaluate_query(queries, dataset_eval, FILE_NAME_PATH):
    result = {'simple': 0, 'medium': 0, 'complex': 0, 'total': 0}

    for q in queries:
        evaluate_result = False
        q['similarity'] = 0
        try:
            if q["query_string"].strip() != "":
                evaluate_result, similarity, column_matching_index = dataset_eval.evaluate_dataset_query(q["query_string"], q["id"], query_type="sql")
                q['column_matching_index'] = column_matching_index
                q['similarity']=similarity
            
        except Exception as e:
            print(str(e))
            
        q['result'] = evaluate_result
        
        if evaluate_result==True:
            result['total'] += 1
            result[q["type"]] += 1
        
    save_queries(queries, FILE_NAME_PATH)   
    return result

def get_file_name(FILE_NAME_PATH):
    parts = FILE_NAME_PATH.split("/")
    last_element = parts[-1]
    file_name = last_element.replace(".json", "")
    return file_name

def get_experiment_name(filename):
    schema = get_schema(filename).upper()
    experiment_name = f"{EXPERIMENT} - {schema}"
    if "chatgpt4" in filename:
        experiment_name += " - GPT-4"
    elif "chatgpt_" in filename:
        experiment_name += " - GPT-3.5"
    elif "llama" in filename:
        experiment_name += " - LLAMA"
    
    if "rag" in filename:
        experiment_name += " - RAG"
        
    if "turbo" in filename:
        experiment_name += " - Turbo"
                        
    if "instances" in filename:
        experiment_name += " - Passing database instances"
        
    if "_fk" in filename:
        experiment_name += " - FK"
        
    return experiment_name


def compute_cost(filename):
    queries = read_queries(filename)
    total_costs = 0
    total_tokens = 0
    input_tokens = 0
    output_tokens = 0
    for q in queries:
        total_tokens += q.get('total_tokens', 0)
        total_costs += q.get('total_cost', 0)
        input_tokens += q.get('prompt_tokens', 0)
        output_tokens += q.get('completion_tokens', 0)

    print(total_tokens)
    print(round(total_costs, 2))
    print(input_tokens, output_tokens)
    return total_tokens, round(total_costs, 2), input_tokens, output_tokens

def compute_time(filename):
    queries = read_queries(filename)
    total_time = 0
    for q in queries:
        total_time += q.get('time', 0)
    print(total_time)
    return str(datetime.timedelta(seconds=round(total_time)))

In [4]:
def run(result_files = [], result_final = [], compute_costs = True):
    
    if result_files == [] :
        result_files = get_files_in_folder()
    else:
        result_files = sorted(result_files)
        
    
    for file in result_files:
        queries = read_queries(file)
        dataset_eval = get_dataset_evaluator(file)
        count_query_type = get_count_query_type(queries)
        result = evaluate_query(queries, dataset_eval, file)
        
        overall_by_type = {}
        for item in result:
            overall_by_type[f"overall_{item}"] = round(result[item] / count_query_type[item], 4)
        
        result['experiment'] = get_file_name(file)
        result = {**result, **overall_by_type}
        result['total_tokens'], result['total_cost'], result['average_tokens_by_query'] = 0, 0, 0
        if compute_costs:
            result['total_tokens'], result['total_cost'], result['input_tokens'], result['output_tokens'] = compute_cost(file)

        result['total_time'] = compute_time(file)
        
        result_final.append(result)
        time.sleep(5)
    return result_final
        

## Validação com o mondial - Foreign Keys

In [9]:
results_fk = run([ 'results/9_sql_queries_chatgpt_mondial_gpt_fk.json',
 'results/10_sql_queries_chatgpt_mondial_gpt_fk_instances.json',
 'results/11_sql_queries_chatgpt4_mondial_gpt_fk_instances.json',
 'results/12_sql_queries_chatgpt4_mondial_gpt_fk.json'], [])

mondial
mondial_gpt
SELECT max(area) FROM mondial_country where name  = 'Thailand'
SELECT area FROM country WHERE name = 'Thailand'
True cols: Index(['MAX(AREA)'], dtype='object')
Adj. True cols: Index(['MAX(AREA)'], dtype='object')
Predicted cols: Index(['AREA'], dtype='object')
Adj. Predicted cols: Index(['AREA'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT name FROM mondial_province where area > 10000
SELECT name
FROM province
WHERE area > 10000
True cols: Index(['NAME'], dtype='object')
Adj. True cols: Index(['NAME'], dtype='object')
Predicted cols: Index(['NAME'], dtype='object')
Adj. Predicted cols: Index(['NAME'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT mondial_language.name FROM mondial_language INNER JOIN mondial_country ON mondial_language.country = mondial_country.code WHERE mondial_country.name = 'Poland'
SELECT name
FROM language
WHERE country = 'PL'
True cols: Index(['NAME'], dtype='object

In [10]:
df = pd.DataFrame(results_fk, columns=['experiment', 'simple', 'medium', 'complex', 'total', 'overall_simple', 'overall_medium', 'overall_complex', 'overall_total','input_tokens', 'output_tokens', 'total_tokens','total_cost', 'total_time'])
df['experiment'] = df['experiment'].apply(get_experiment_name)
df = df.sort_values(by='overall_total', ascending=False)
df

,experiment,simple,medium,complex,total,overall_simple,overall_medium,overall_complex,overall_total,input_tokens,output_tokens,total_tokens,total_cost,total_time
1,SQLQueryChain - MONDIAL_GPT - GPT-4 - Passing ...,27,28,15,70,0.8182,0.8485,0.4412,0.70,782624,2527,785151,23.63,0:09:20
2,SQLQueryChain - MONDIAL_GPT - GPT-4 - FK,27,26,16,69,0.8182,0.7879,0.4706,0.69,569713,2643,572356,17.25,0:05:28
0,SQLQueryChain - MONDIAL_GPT - GPT-3.5 - Passin...,25,22,13,60,0.7576,0.6667,0.3824,0.60,782813,2501,785314,2.36,0:06:47
3,SQLQueryChain - MONDIAL_GPT - GPT-3.5 - FK,25,19,11,55,0.7576,0.5758,0.3235,0.55,569713,2563,572276,1.72,0:02:15


In [11]:
df.to_csv(TARGET_PATH_FK)  

## Validação com o mondial - Foreign Keys + RAG

In [5]:
results_fk = run(['results/13_sql_queries_rag_chatgpt_mondial_gpt.json',
                  'results/14_sql_queries_rag_chatgpt4_mondial_gpt.json'], [])

mondial
mondial_gpt
SELECT max(area) FROM mondial_country where name  = 'Thailand'
SELECT area FROM mondial_country WHERE name = 'Thailand'
True cols: Index(['MAX(AREA)'], dtype='object')
Adj. True cols: Index(['MAX(AREA)'], dtype='object')
Predicted cols: Index(['AREA'], dtype='object')
Adj. Predicted cols: Index(['AREA'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT name FROM mondial_province where area > 10000
SELECT name
FROM mondial_province
WHERE area > 10000;
True cols: Index(['NAME'], dtype='object')
Adj. True cols: Index(['NAME'], dtype='object')
Predicted cols: Index(['NAME'], dtype='object')
Adj. Predicted cols: Index(['NAME'], dtype='object')
True: query matches sql (Similarity: 1.0, column match: 1.0)
SELECT mondial_language.name FROM mondial_language INNER JOIN mondial_country ON mondial_language.country = mondial_country.code WHERE mondial_country.name = 'Poland'
SELECT l.name
FROM mondial_language l
INNER JOIN mondial_country c ON 

In [6]:
df_rag = pd.DataFrame(results_fk, columns=['experiment', 'simple', 'medium', 'complex', 'total', 'overall_simple', 'overall_medium', 'overall_complex', 'overall_total','input_tokens', 'output_tokens', 'total_tokens','total_cost', 'total_time'])
df_rag['experiment'] = df_rag['experiment'].apply(get_experiment_name)
df_rag = df_rag.sort_values(by='overall_total', ascending=False)
df_rag

,experiment,simple,medium,complex,total,overall_simple,overall_medium,overall_complex,overall_total,input_tokens,output_tokens,total_tokens,total_cost,total_time
1,SQLQueryChain - MONDIAL_GPT - GPT-4 - RAG,28,27,25,80,0.8485,0.8182,0.7353,0.80,243519,3943,247462,0.00,0:05:38
0,SQLQueryChain - MONDIAL_GPT - GPT-3.5 - RAG,25,20,17,62,0.7576,0.6061,0.5000,0.62,243508,3338,246846,0.37,0:02:19
